In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
import glob

In [5]:
#tb estabelecimento 2023 mes 3
cnes = spark.read.csv("/dados10t/datalake/raw/omop/dados_auxiliares_de_input/cnes2023_03/tbEstabelecimento202302.csv",sep=";",header=True,inferSchema=False)
print(cnes.count())

482479


In [6]:
#tb estabelecimento 2023 mes 3
#cnes = spark.read.csv("/dados10t/datalake/standard/omop/dados_auxiliares_de_input/cnes2023_03/tbEstabelecimento202302.csv",sep=";",header=True,inferSchema=False)
#print(cnes.count())

In [7]:
cnes.select("TP_UNIDADE").take(2)

[Row(TP_UNIDADE='02'), Row(TP_UNIDADE='40')]

In [8]:
cnes.printSchema()

root
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_CNES: string (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: string (nullable = true)
 |-- TP_PFPJ: string (nullable = true)
 |-- NIVEL_DEP: string (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: string (nullable = true)
 |-- CO_CEP: string (nullable = true)
 |-- CO_REGIAO_SAUDE: string (nullable = true)
 |-- CO_MICRO_REGIAO: string (nullable = true)
 |-- CO_DISTRITO_SANITARIO: string (nullable = true)
 |-- CO_DISTRITO_ADMINISTRATIVO: string (nullable = true)
 |-- NU_TELEFONE: string (nullable = true)
 |-- NU_FAX: string (nullable = true)
 |-- NO_EMAIL: string (nullable = true)
 |-- NU_CPF: string (nullable = true)
 |-- NU_CNPJ: string (nullable = true)
 |-- CO_ATIVIDADE: string (nullable = true)
 |-- CO_CLIENTELA: string (nullab

In [9]:
municipios_br = spark.read.csv("/home/juracy.santos/dados/lista_municipios_brasil.csv",sep=",",header=True,inferSchema=False)
print(municipios_br.count())
municipios_br_v2 = municipios_br.select("codigo","codigo_6")

5571


In [10]:
municipios_br_v2.printSchema()

root
 |-- codigo: string (nullable = true)
 |-- codigo_6: string (nullable = true)



In [11]:
cnes = cnes.join(municipios_br_v2, cnes.CO_MUNICIPIO_GESTOR == municipios_br_v2.codigo_6,'left') # 350 nao da match com o codigo de 7 digitos e ficam sem o id do municipio

In [13]:
cnes.count()

482479

In [14]:
cnes.select("CO_CNES").distinct().count()

482479

In [15]:
cnes.select("CO_UNIDADE").distinct().count()

482479

In [36]:
#tb estabelecimento 2023 mes 3 - qual usar para o tipo do estabelecimento? tb tipo estabelecimento  ou tipo unidade ? segundo a documentacao ta tb estabelecimento seria tb_unidade, onde entra tbtipoestabelcimento?
#tipo_estab = spark.read.csv("/dados10t/datalake/raw/omop/dados_auxiliares_de_input/cnes2023_03/tbTipoEstabelecimento202302.csv",sep=";",header=True,inferSchema=False)
#print(tipo_estab.count())
#print(tipo_estab.show(50,False)) # ao final usei a tabela tp unidade(tbTipoUnidade202302)

In [37]:
#tb estabelecimento 2023 mes 3
tipo_unidade = spark.read.csv("/dados10t/datalake/raw/omop/dados_auxiliares_de_input/cnes2023_03/tbTipoUnidade202302.csv",sep=";",header=True,inferSchema=False)

In [39]:
tipo_unidade.orderBy("CO_TIPO_UNIDADE").show(200,False)

+---------------+------------------------------------------------------------+
|CO_TIPO_UNIDADE|DS_TIPO_UNIDADE                                             |
+---------------+------------------------------------------------------------+
|01             |POSTO DE SAUDE                                              |
|02             |CENTRO DE SAUDE/UNIDADE BASICA                              |
|04             |POLICLINICA                                                 |
|05             |HOSPITAL GERAL                                              |
|07             |HOSPITAL ESPECIALIZADO                                      |
|15             |UNIDADE MISTA                                               |
|20             |PRONTO SOCORRO GERAL                                        |
|21             |PRONTO SOCORRO ESPECIALIZADO                                |
|22             |CONSULTORIO ISOLADO                                         |
|32             |UNIDADE MOVEL FLUVIAL              

In [20]:
tipo_unidade.count()

39

In [21]:
tipo_unidade.select("CO_TIPO_UNIDADE").distinct().count()

39

In [22]:
cnes.count()

482479

In [56]:
map_care_site = spark.read.csv("/dados10t/datalake/standard/omop/resultado_mapeamento/care_site_mapping.csv",sep=",",inferSchema=False,header=True)

In [57]:
map_care_site = map_care_site.withColumn("id",F.split("source_code",";")[0])
map_care_site = map_care_site.withColumn("CO_TIPO_UNIDADE",F.regexp_replace("id",'\"',"") )
map_care_site_v2 = map_care_site.select("CO_TIPO_UNIDADE","target_concept_id")

In [60]:
tipo_unidade_map = tipo_unidade.join(map_care_site_v2,['CO_TIPO_UNIDADE'],'inner')

In [62]:
tipo_unidade_map.count()

39

In [63]:
tipo_unidade_map.show(10,False)

+---------------+---------------------------------------------------------+-----------------+
|CO_TIPO_UNIDADE|DS_TIPO_UNIDADE                                          |target_concept_id|
+---------------+---------------------------------------------------------+-----------------+
|69             |CENTRO DE ATENCAO HEMOTERAPIA E OU HEMATOLOGICA          |38004240         |
|70             |CENTRO DE ATENCAO PSICOSSOCIAL                           |8964             |
|71             |CENTRO DE APOIO A SAUDE DA FAMILIA                       |38004247         |
|72             |UNIDADE DE ATENCAO A SAUDE INDIGENA                      |38004702         |
|01             |POSTO DE SAUDE                                           |38004247         |
|02             |CENTRO DE SAUDE/UNIDADE BASICA                           |38004247         |
|04             |POLICLINICA                                              |38004247         |
|22             |CONSULTORIO ISOLADO                        

In [65]:
tipo_unidade_map = tipo_unidade_map.withColumnRenamed("CO_TIPO_UNIDADE","TP_UNIDADE")

In [66]:
cnes_v2 = cnes.join(tipo_unidade_map,['TP_UNIDADE'],'left') #perde-se 1 unidade usando inner , capturar cnes desse 1 faltante

In [67]:
cnes_v2.count()

482479

In [68]:
cnes_v2.printSchema()

root
 |-- TP_UNIDADE: string (nullable = true)
 |-- CO_UNIDADE: string (nullable = true)
 |-- CO_CNES: string (nullable = true)
 |-- NU_CNPJ_MANTENEDORA: string (nullable = true)
 |-- TP_PFPJ: string (nullable = true)
 |-- NIVEL_DEP: string (nullable = true)
 |-- NO_RAZAO_SOCIAL: string (nullable = true)
 |-- NO_FANTASIA: string (nullable = true)
 |-- NO_LOGRADOURO: string (nullable = true)
 |-- NU_ENDERECO: string (nullable = true)
 |-- NO_COMPLEMENTO: string (nullable = true)
 |-- NO_BAIRRO: string (nullable = true)
 |-- CO_CEP: string (nullable = true)
 |-- CO_REGIAO_SAUDE: string (nullable = true)
 |-- CO_MICRO_REGIAO: string (nullable = true)
 |-- CO_DISTRITO_SANITARIO: string (nullable = true)
 |-- CO_DISTRITO_ADMINISTRATIVO: string (nullable = true)
 |-- NU_TELEFONE: string (nullable = true)
 |-- NU_FAX: string (nullable = true)
 |-- NO_EMAIL: string (nullable = true)
 |-- NU_CPF: string (nullable = true)
 |-- NU_CNPJ: string (nullable = true)
 |-- CO_ATIVIDADE: string (nullable

In [69]:
tb_care_site = cnes_v2.select("CO_CNES","DS_TIPO_UNIDADE","NO_FANTASIA","codigo","target_concept_id") #CO_MUNICIPIO_GESTOR removido agora usamos codigo que tem 7 digitos e nao CO_MUNICIPIO_GESTOR que só tem 6

In [98]:
#tb_care_site.show(500,False)

In [70]:
tb_care_site_v2 = tb_care_site.withColumnRenamed("CO_CNES","care_site_id")
tb_care_site_v2 = tb_care_site_v2.withColumnRenamed("NO_FANTASIA","care_site_name")
tb_care_site_v2 = tb_care_site_v2.withColumnRenamed("codigo","location_id")
tb_care_site_v2 = tb_care_site_v2.withColumnRenamed("DS_TIPO_UNIDADE","place_of_service_source_value")
tb_care_site_v2 = tb_care_site_v2.withColumn("care_site_source_value",F.col("care_site_id"))
tb_care_site_v2 = tb_care_site_v2.withColumnRenamed("target_concept_id","place_of_service_concept_id")


#tb_care_site_v2 = tb_care_site_v2.withColumn("place_of_service_concept_id",F.lit(""))

In [71]:
tb_care_site_v2 = tb_care_site_v2.select("care_site_id","care_site_name","place_of_service_concept_id","location_id","care_site_source_value","place_of_service_source_value")

In [73]:
tb_care_site_v2.show(5,False)

+------------+--------------------------+---------------------------+-----------+----------------------+------------------------------+
|care_site_id|care_site_name            |place_of_service_concept_id|location_id|care_site_source_value|place_of_service_source_value |
+------------+--------------------------+---------------------------+-----------+----------------------+------------------------------+
|2569302     |USF BULTRINS MONTE II     |38004247                   |2609600    |2569302               |CENTRO DE SAUDE/UNIDADE BASICA|
|2571943     |UNIDADE MOVEL             |38004247                   |2609600    |2571943               |UNIDADE MOVEL TERRESTRE       |
|2344637     |USF ALTO DA MINA          |38004247                   |2609600    |2344637               |CENTRO DE SAUDE/UNIDADE BASICA|
|2344696     |USF ILHA DE SANTANA I E II|38004247                   |2609600    |2344696               |CENTRO DE SAUDE/UNIDADE BASICA|
|2142295     |HOSPITAL CARLOS CHAGAS    |3800451

In [74]:
tb_care_site_v2.toPandas().to_csv("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_care_site.csv",sep=",",index=False)

In [76]:
tb_care_site_v2.write.parquet("/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_care_site.parquet")